In [ ]:
# default_exp core

In [ ]:
#export
import threading
import time
from torch.autograd.profiler import FunctionEvent, EventList
from fastai2.callback.core import Callback
import json

class ChromeTrace(Callback):
    def __init__(self, ll):
        super().__init__()
        self.l = ll
        self.d = {}
    def __call__(self, event_name):
        tid = threading.get_ident()
        tname = threading.current_thread().name
        if event_name.startswith('begin'):
            evt = event_name.replace('begin_', '')
            self.d[evt] = time.process_time()*1000_000
        elif event_name.startswith('after'):
            evt = event_name.replace('after_', '')
            end = time.process_time()*1000_000
            if evt in self.d:
                start = self.d[evt]
                fe = FunctionEvent(tname, tid, evt, tid, start, end)
            else:
                # hack if there is only after_xxx, we need a start 500, 1000, 5000?
                start = end-500 # 1-time after_xxx
                fe = FunctionEvent(tname, tid, evt, tid, start, end)
            self.l.append(fe)
            self.d.pop(evt, None)
    def export_chrome_trace(self, fname="export_chrome_trace.json", dumps=False):
        with open(fname, mode="w+") as f:
            prof = EventList(self.l)
            prof.export_chrome_trace(f.name)
            if dumps:
                parsed = json.load(f)
                print(json.dumps(parsed, indent=4, sort_keys=True))